Daten vervollständigen / anreichern

- Ertrag pro m'2 pro Gemeinde/Kanton 
- Summe der Dachflächen pro Gemeinde / Kanton 
- Summe Ertrag pro Gemeinde / Kanton 
- Koordinanten umrechnen (LV95 zu lat/lon)

In [ ]:
# Dachinfo.csv Datei laden

In [2]:
import pandas as pd

df = pd.read_csv('Data/DachInfo.csv')


In [ ]:
# Geladene Daten prüfen, erste Zeilen ausgeben

In [ ]:
print(df.head())


In [ ]:
# Summe Dachflächen pro Gemeinde (GGDENAME) (Daten: Gemeinde, Kanton, Summe Total Dachfläche) in CSV exportieren

In [4]:
# Gruppieren der Daten nach Gemeinde (GGDENAME) und Kanton (GDEKT) und Berechnung Summe der Dachflächen pro Gemeinde

df_gemeinden = df.groupby(['GGDENAME', 'GDEKT'])['Summe_Flaeche'].sum().reset_index()



In [ ]:
# df_gemeinden erste Zeilen prüfen ob Daten richtig erstellt wurden

print (df_gemeinden.head())

In [ ]:
# Speichern des Ergebnis in eine CSV-Datei

df_gemeinden.to_csv('data/summe_dachflaechen_pro_gemeinde.csv', index=False)

In [ ]:
# Summe Ertrag pro Gemeinde (GGDENAME) (Spalten: Gemeinde, Kanton, Summe Total Ertrag) in CSV exportieren

In [6]:
# Gruppieren der Daten nach Gemeinde (GGDENAME) und Kanton (GDEKT) und Berechnung Summe des Ertrags pro Gemeinde
df_ertrag_gemeinden = df.groupby(['GGDENAME', 'GDEKT'])['Summe_Stromertrag'].sum().reset_index()


In [ ]:
# df_ertrag_gemeinden erste Zeilen prüfen ob Daten richtig erstellt wurden
print (df_ertrag_gemeinden.head())

In [7]:
# Speichern des Ergebnis in eine CSV-Datei
df_ertrag_gemeinden.to_csv('Data/summe_ertrag_pro_gemeinde.csv', index=False)


In [ ]:
# Berechnung welche Gemeinde den höchsten Ertrag pro Quadratmeter hat: df_gemeinden und df_ertrag_gemeinden zusammenführen und neue Spalte hinzufügen, die den Ertrag pro Quadratmeter berechnet.

In [9]:
# DataFrames df_gemeinden und df_ertrag_gemeinden zusammenfühhren, um ein neues DataFrame df_combined zu erstellen
# Mit pd.merge() die beiden DataFrames df_gemeinden und df_ertrag_gemeinden kombinieren basierend auf den gemeinsamen Spalten 'GGDENAME' und 'GDEKT' 
# und ein neues DataFrame df_combined erstellen


df_combined = pd.merge(df_gemeinden, df_ertrag_gemeinden, on=['GGDENAME', 'GDEKT'])


In [ ]:
# Neues DataFrame df_combined überprüfen, erste Zeilen ausgeben

print(df_combined.head())

In [11]:
# Berechnung neue Spalte mit dem Ertrag pro Quadratmeter
df_combined['Ertrag_pro_Quadratmeter'] = df_combined['Summe_Stromertrag'] / df_combined['Summe_Flaeche']


In [ ]:
# Neues DataFrame df_combined überprüfen, erste Zeilen ausgeben

print(df_combined.head())

In [14]:
# Optional: Das DataFrame df_combined nach der neuen Spalte Ertrag_pro_Quadratmeter in absteigender Reihenfolge sortieren

df_combined = df_combined.sort_values('Ertrag_pro_Quadratmeter', ascending=False)


In [ ]:
# Optional: Neues DataFrame df_combined (sortiert Absteigend nach Ertrag_pro_Quadratmeter) überprüfen, erste Zeilen ausgeben

print(df_combined.head())

In [13]:
# Das Ergebnis in eine CSV-Datei mit der to_csv()-Methode speichern
df_combined.to_csv('Data/ertrag_pro_quadratmeter_pro_gemeinde.csv', index=False)


Funktion für Umrechung LV95 to lat/lon Koordinaten

In [5]:
from pyproj import Transformer

def lv95_to_latlon(easting, northing):
    transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326")  # LV95 (EPSG:2056) zu WGS84 (EPSG:4326)
    lon, lat = transformer.transform(easting, northing)
    return lon, lat


Funktion testen

In [ ]:
east = 2676067.5
north = 1178574.6

lat, lon = lv95_to_latlon(east, north)
print(f"Latitude: {lat}, Longitude: {lon}")


Berechung von Gebäudekoordinaten lv95 --> lat/lon

In [8]:
import pandas as pd

# CSV-Datei einlesen
data = pd.read_csv('data/DachInfo.csv')

# Neue Spalte erstellen und Berechnungen durchführen
data['Koord_lat'], data['Koord_log'] = lv95_to_latlon(data['GKODE'],data['GKODN'])

# Ergebnisse in einer neuen CSV-Datei speichern
data.to_csv('data/DachInfo4View.csv', index=False)